In [5]:
import json, logging, os, re, requests, sys, time
import pandas as pd

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from datetime import date, datetime
from pykrx import stock
from webob.compat import urlparse
import lxml
URL_NAVER_FINANCE_NEWS_QUERY = "https://finance.naver.com/news/news_search.nhn?rcdate=&q=%s&x=0&y=0&sm=all.basic&pd=4&stDateStart=%s&stDateEnd=%s&page=%s"
URL_NAVER_FINANCE = "http://finance.naver.com"

In [36]:
pip install webob
pip install pykrx
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [8]:
class NaverFinanceNewsCrawler:
    def __init__(self):
        self.ticker = None
        self.result = []
        pass

    def _crawl_by_query(self, ticker, dt):
        """
        Crawl Naver Finance News
        :param ticker: string; search keywords
        :return: generator; [{title, summary, url, articleId, content, codes}, ...]
        """
        comp_name = stock.get_market_ticker_name(ticker)
        print("comp name = ", comp_name)
        # Convert the query to euc-kr string
        q = ""
        for c in comp_name.encode("euc-kr"):
            q += "%%%s" % format(c, "x").capitalize()

        # 여러 페이지를 동시에 돌려주므로, 기사가 존재할 때까지 loop 돌면서 뉴스 가져오기
        page = 1
        n_news = 0

        while True:
            r_url = URL_NAVER_FINANCE_NEWS_QUERY % (q, dt, dt, page,)
            print('url = ', r_url)
            r = requests.get(r_url)
            soup = bs(r.text, "lxml")
            news = soup.find("div", class_="newsSchResult").find(
                "dl", class_="newsList"
            )
            news_title = news.find_all("dd", class_="articleSubject")
            news_summary = news.find_all("dd", class_="articleSummary")
            wdate = news.find_all("span", class_="wdate")
            n_news += len(news_title)

            if len(news_title) > 0:
                for title, summary, _date in zip(news_title, news_summary, wdate):
                    date = _date.find(text=True).split("\n")[2].strip()

                    if date == dt:
                        url = URL_NAVER_FINANCE + title.a.get("href")
                        res = {
                            "title": title.a.text,
                            "summary": summary.find(text=True).strip(" \t\n\r"),
                            "url": url,
                            "ticker": ticker,
                            "articleId": urlparse.parse_qs(
                                urlparse.urlparse(url).query
                            )["article_id"][0],
                            "date": date,
                        }
                        res.update(self._crawl_content(url))
                        #             if self.query in res['title']:
                        self.result.append(res)
                        time.sleep(1)
                page += 1

            else:
                break
        print("number of news for comp {} = {}".format(comp_name, n_news))

    def _crawl_content(self, url):
        r = requests.get(url)
        soup = bs(r.text, "lxml")
        content = soup.find("div", id="content", class_="articleCont")
        codes = re.findall(r"\d{6}", content.text)
        cntnt = content.text.strip(" \t\n\r").split("@")[0]
        # 마지막 마침표 이후 제거
        effix = cntnt.split('.')[-1]
        cntnt = cntnt.replace(effix, '')
        return {"content": cntnt}

    def get_valid_ticker_name(self, ticker):
        try:
            return stock.get_market_ticker_name(ticker)
        except:
            pass

    def get_news(self, ticker, dt):
        # logger instance 생성
        logger = logging.getLogger(__name__)
        # Format 설정
        formatter = logging.Formatter(
            "%(asctime)s %(levelname)s [%(name)s] [%(filename)s:%(lineno)d] - %(message)s"
        )
        # handler 생성
        streamHandler = logging.StreamHandler()
        streamHandler.setFormatter(formatter)
        logger.addHandler(streamHandler)
        logger.setLevel(level=logging.INFO)

        today = date.today().strftime("%Y-%m-%d")
        print(f'today= {today}')
        if dt is None:
            dt = today
        else:
            dt = dt

        # 회사 정보 만들기.
        _comp = {}
        _comp["ticker"] = ticker
        _comp["name"] = [self.get_valid_ticker_name(x) for x in ticker]
    
        print('_comp = ', _comp)
        # 이미 있는 날짜와 회사 정보 가져오기

        for i, t in enumerate(_comp["ticker"]):
            logger.info("i = {}, comp = {}".format(i, _comp["ticker"][i]))
            try:
                msg = "company = {}".format(_comp["name"][i])
                logger.info(msg)
                time.sleep(1)
                self._crawl_by_query(ticker=t, dt=dt)
            except Exception as e:
                logger.error(e)
                pass


In [9]:
NNC = NaverFinanceNewsCrawler()
res = NNC.get_news(['005930', '005930'], '2022-03-22')

2022-03-26 06:39:56,523 INFO [__main__] [<ipython-input-8-6a27da70f6c1>:110] - i = 0, comp = 005930
2022-03-26 06:39:56,523 INFO [__main__] [<ipython-input-8-6a27da70f6c1>:110] - i = 0, comp = 005930
2022-03-26 06:39:56,523 INFO [__main__] [<ipython-input-8-6a27da70f6c1>:110] - i = 0, comp = 005930
2022-03-26 06:39:56,525 INFO [__main__] [<ipython-input-8-6a27da70f6c1>:113] - company = 삼성전자
2022-03-26 06:39:56,525 INFO [__main__] [<ipython-input-8-6a27da70f6c1>:113] - company = 삼성전자
2022-03-26 06:39:56,525 INFO [__main__] [<ipython-input-8-6a27da70f6c1>:113] - company = 삼성전자


today= 2022-03-26
_comp =  {'ticker': ['005930', '005930'], 'name': ['삼성전자', '삼성전자']}
comp name =  삼성전자
url =  https://finance.naver.com/news/news_search.nhn?rcdate=&q=%Bb%Ef%Bc%Ba%C0%Fc%C0%Da&x=0&y=0&sm=all.basic&pd=4&stDateStart=2022-03-22&stDateEnd=2022-03-22&page=1
url =  https://finance.naver.com/news/news_search.nhn?rcdate=&q=%Bb%Ef%Bc%Ba%C0%Fc%C0%Da&x=0&y=0&sm=all.basic&pd=4&stDateStart=2022-03-22&stDateEnd=2022-03-22&page=2
url =  https://finance.naver.com/news/news_search.nhn?rcdate=&q=%Bb%Ef%Bc%Ba%C0%Fc%C0%Da&x=0&y=0&sm=all.basic&pd=4&stDateStart=2022-03-22&stDateEnd=2022-03-22&page=3


KeyboardInterrupt: 

In [12]:
NNC.result

[{'title': '삼성SDS 급락 왜… 총수 일가 상속세 마련 주식 처분한 듯',
  'summary': '홍라희 전 리움미술관장도 지난해 10월 KB국민은행과 삼성전자 주식 1994만 1860주(0.33%)에 대해 상속세 납부 목적으로 처분 신탁 계...',
  'url': 'http://finance.naver.com/news/news_read.naver?article_id=0003260069&office_id=081&mode=search&query=삼성전자&page=1',
  'ticker': '005930',
  'articleId': '0003260069',
  'date': '2022-03-22',
  'content': '주식 3900억 시간 외 대량매매이부진·서현 자매 물량과 일치삼성SDS 사옥 전경.삼성 총수 일가가 계열사인 삼성SDS 주식 3900여억원어치를 시간 외 대량매매(블록딜)로 처분한다는 소식에 22일 삼성SDS 주가가 급락했다. 특히 삼성SDS는 장 초반에 전 거래일보다 8.93%(1만 2500원) 하락한 12만 7500원까지 밀리며 52주 신저가를 썼다. 주가는 전날보다 7.14%(1만원) 떨어진 13만원으로 거래를 마쳤다.금융투자 업계에 따르면 KB증권과 모건스탠리는 전날 장 마감 직후 삼성SDS 보통주 301만 8860주(3.90%)를 매각하기 위한 수요 예측에 나섰다. 매각 가격은 전날 종가인 14만원에서 8.8% 할인한 것으로 알려졌다.재계에서는 이번 삼성 계열사 지분 매각이 이부진 호텔신라 대표이사(사장)와 이서현 삼성복지재단 이사장의 상속세 재원 마련을 위한 것으로 보고 있다. 지난해 10월 이부진 사장과 이서현 이사장은 KB국민은행과 각각 삼성SDS 주식 150만 9430주의 매각 신탁 계약을 체결했다. 이번에 대량매매로 나온 물량은 두 자매의 물량을 합친 것과 일치한다. 매각 처분 시한은 오는 4월 25일까지였다. 홍라희 전 리움미술관장도 지난해 10월 KB국민은행과 삼성전자 주식 1994만 1860주(0.33%